In [69]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np

#ML models 
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, IsolationForest
import xgboost as xgb
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

# Models
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier

#preprocessing
from sklearn.model_selection import train_test_split 

# TDA magic
import gtda.time_series as ts
from gtda.homology import EuclideanCechPersistence
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import PairwiseDistance
from gtda.diagrams import PersistenceEntropy



#data visualization 
import plotly.graph_objects as go
import plotly.express as px
from matplotlib import pyplot
from pandas.plotting import lag_plot
from pandas.plotting import autocorrelation_plot
import matplotlib.pyplot as plt
import seaborn as sns

#statistical analysis
from statsmodels.tsa.stattools import adfuller


# Financial data 
from talib import RSI
import ta
import joblib

In [76]:
df = pd.read_csv('X_test.csv')
#set date as index
df['Date'] = pd.DatetimeIndex(df['Date'])
df.set_index('Date',inplace=True)


# Load the model from the file
loaded_rf_model = joblib.load('best_rf_model_GBPUSD=X.sav')

In [77]:
dollar = 1000 #how much dollar we have 
n_stock = 0 #how much of this stock we have
net_worth_list = [] #networth of our model in each step 
holding = False #this shows if we buy stock and we need to sell it next week or not(task constrint)
risk_index = 0.52 #the higher value of this we will decrease the risk of trading (it's threshold for model probablity)

In [78]:
#our model 
for i in range(len(df)):
    curr_action = 0  #0 nutral, 1 buy , 2 sell 
    
    #price at this step 
    price = df[i:i+1]['real_close'].values[0]
    
    #using model  
    probs = loaded_rf_model.predict_proba(df[i:i+1].drop('real_close',axis=1))
    #confidence of model 
    prob_val = np.max(probs)
    class_val = np.argmax(probs)
    
    #storing networth 
    net_worth_list.append(dollar+n_stock*price)
    
    #confident trading 
    if prob_val > risk_index:
        
        if class_val == 1 and not holding: 
            #buying
            n_stock += int(dollar / price)
            dollar -= n_stock * price
            holding = True
            curr_action = 1
            
        elif holding:
            #selling
            dollar += n_stock * price
            n_stock = 0
            holding = False
            curr_action = 2
    if holding and curr_action == 0:
            #selling 
            dollar += n_stock * price
            n_stock = 0
            holding = False
            curr_action = 2
        
            
        

## Buy and hold strategy:
To benchmark our model we use Buy and hold strategy here.

In [79]:
#assuming we change all of our dollar to stock and number of stocks should be quantized
n_stock = int(1000/df.iloc[0]['real_close'])

In [80]:
#buy and hold networth
net_worth_list_benchmark = []
for i in range(len(df)):
    price = df[i:i+1]['real_close'].values[0]
    net_worth_list_benchmark.append(n_stock*price)

# Visualization

In [81]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = df.index, y = net_worth_list_benchmark,mode='lines',name = 'buy and hold'))
fig.add_trace(go.Scatter(x = df.index, y = net_worth_list,mode='lines',name = 'model'))

fig.show()

# Results - May 2022 to Jan 2024 
**GBPUSD=X**
- risk_index = 0.52: our model return = 103.22%, Benchmarked model return = 102.17%
- risk_index = 0.50: our model return = 115.7%, Benchmarked model return = 102.17%
